In [1]:
import pandas as pd
import torch
# from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split



In [2]:
  from google.colab import files

  uploaded = files.upload()

Saving train_data1.xlsx to train_data1.xlsx


In [3]:
# Load the Excel dataset
data = pd.read_excel('train_data1.xlsx')  # Update the path to your dataset

# Assuming your Excel has 'source_text' and 'target_text' columns for input-output pairs
# Preprocess the dataset and split it into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Rename columns to match source and tRRarget text
# data = data.rename(columns={'Yoruba': 'source_text', 'Ena': 'target_text'})

# Create a Hugging Face dataset from the DataFrame
# dataset = Dataset.from_pandas(data)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_text  1490 non-null   object
 1   target_text  1490 non-null   object
 2   English      24 non-null     object
dtypes: object(3)
memory usage: 35.0+ KB


In [5]:
data.describe

<bound method NDFrame.describe of                                             source_text  \
0     eyan kan fun jenifa ni leta ife pelu ebun egba...   
1                                          o ma loki o.   
2                            se o ti ni afesona nisiyi?   
3                                won ti e ku ri ewa wo.   
4                                    jowo ma pamilerin.   
...                                                 ...   
1485                ta gbọn pe wọn ti ji ẹni egbere lọ.   
1486  nigba fi eniyan ta buru bayii ba n ronu ko jaw...   
1487                awọn osere aṣọ hihun naa rẹwa pupọ.   
1488                           ti lo fẹ ki n ṣe fun sẹ?   
1489                              wahala rẹ poju ni mo.   

                                            target_text  \
0     egeyanga kanga fungun jegenigifaga nigi legeta...   
1                                ogo maga logokigi ogo.   
2      sege ogo tigi nigi agafegesogonaga nigisigiyigi?   
3               wongan tigi ege kugu rigi egewaga wogo.   
4                       jojowogo maga pagamigilegerigi.   
...                                                 ...   
1485  taga gbọngọn pege wọngọn tigi jigi ẹgẹnigi ege...   
1486  nigigbaga figi egenigiyangan taga bugurugu bag...   
1487  agawọngọn ogosegerege agaṣọgọ higihungun nagaa...   
1488           tigi logo fẹgẹ kigi ng ṣege fungun sẹgẹ?   
1489              wagahagalaga rẹgẹ pogojugu nigi mogo.   

                                                English  
0     Someone gave Jennifer a love letter, a bracele...  
1                                         She is lucky.  
2                              She has a lover already?  
3                              No one even looks at us.  
4                          Please do not make me laugh.  
...                                                 ...  
1485                                                NaN  
1486                                                NaN  
1487                                                NaN  
1488                                                NaN  
1489                                                NaN  

[1490 rows x 3 columns]>

In [6]:
data.head()

,source_text,target_text,English
0,eyan kan fun jenifa ni leta ife pelu ebun egba...,egeyanga kanga fungun jegenigifaga nigi legeta...,"Someone gave Jennifer a love letter, a bracele..."
1,o ma loki o.,ogo maga logokigi ogo.,She is lucky.
2,se o ti ni afesona nisiyi?,sege ogo tigi nigi agafegesogonaga nigisigiyigi?,She has a lover already?
3,won ti e ku ri ewa wo.,wongan tigi ege kugu rigi egewaga wogo.,No one even looks at us.
4,jowo ma pamilerin.,jojowogo maga pagamigilegerigi.,Please do not make me laugh.


In [7]:
data.shape

(1490, 3)

In [8]:
train_data.shape

(1192, 3)

In [9]:
test_data.shape

(298, 3)

In [10]:
# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:

# Function to tokenize the dataset
def preprocess_function(examples):
    # Iterate directly over the DataFrame rows
    inputs = list(examples['source_text'])
    targets = list(examples['target_text'])

    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(targets, max_length=512, padding="max_length", truncation=True).input_ids

    # Replace padding token id's of the labels by -100 so it's ignored by the loss function
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in label_ids] for label_ids in labels]
    model_inputs["labels"] = labels

    return model_inputs

# Apply preprocessing
# encoded_dataset = dataset.map(preprocess_function, batched=True)

In [12]:
# Preprocess the data for both training and testing sets
train_dataset = preprocess_function(data)
test_dataset = preprocess_function(test_data)


In [13]:
# Convert to PyTorch dataset format
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.data.items()}



In [14]:
# Load the preprocessed dataset
train_dataset = CustomDataset(train_dataset)
test_dataset = CustomDataset(test_dataset)

In [15]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Increase batch size if you have enough memory
    per_device_eval_batch_size=16,   # Adjust similarly for evaluation
    num_train_epochs=5000,              # Reduce epochs for faster training
    weight_decay=0.01,
    save_strategy="steps",
    save_steps=500,                  # Save frequently to avoid losing progress
    logging_dir='./logs',
    logging_steps=10,               # Log every 100 steps
    max_steps=5000,                  # Stop after a specific number of steps
    fp16=True                        # Enable mixed precision if supported
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
10,4.317900,4.017490
20,4.026600,3.744428
30,3.823000,3.493831
40,3.709200,3.280159
50,3.538600,3.101845
60,3.149100,2.925925
70,3.143200,2.754776
80,2.953700,2.589607
90,2.837000,2.455676
100,2.708900,2.342954


In [ ]:
import shutil
from google.colab import files

# Step 1: Save model and tokenizer
output_dir = "./trained_model"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# Step 2: Compress the saved model directory
shutil.make_archive("trained_model", 'zip', output_dir)

# Step 3: Download the zip file
files.download("trained_model.zip")


In [ ]:
# Evaluation
trainer.evaluate()

{'eval_loss': 0.31681787967681885,
 'eval_runtime': 3.2927,
 'eval_samples_per_second': 62.562,
 'eval_steps_per_second': 3.948,
 'epoch': 96.15384615384616}

In [ ]:
def translate_text(input_text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move the model to the appropriate device

    # Tokenize input and move to the same device as the model
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generate output and move back to CPU for decoding
    outputs = model.generate(input_ids=inputs["input_ids"], max_length=512)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return translated_text


In [ ]:

# Example usage
input_sentence = "Translate Yoruba to Ena: ade je ra bata l'ojo"
translated_sentence = translate_text(input_sentence)
print("Translated Sentence:", translated_sentence)

Translated Sentence: gruba togo enaga: agadege jege raga batataga l'ogojogo


In [ ]:

from transformers import T5ForConditionalGeneration

# Load the trained model
model = T5ForConditionalGeneration.from_pretrained("./trained_model")


In [ ]:
  from google.colab import files

  uploaded = files.upload()

Saving test_data.xlsx to test_data (1).xlsx


In [ ]:
test_data1 = pd.read_excel('test_data (1).xlsx')

In [ ]:
translated_sentences = []
for item in test_dataset:
    translated_sentence = translate_text(item["source_text"])  # Your translation function
    translated_sentences.append(translated_sentence)


KeyError: 'source_text'

In [ ]:
test_data1.head()

,source_text,target_text
0,won ta ofa fun jagunjagun naa laya.,wongon taga ogofaga fungun jagagungunjagagungu...
1,o ni imo pupo ni opolo.,ogo nigi igimogo pugupogo nigi ogopogologo.
2,idi ti mo fi feran aye oko ni pe gbogbo nnkan ...,igidigi tigi mogo figi fegerangan agayege ogok...
3,"mama mi, mo ti yo.","magamaga migi, mogo tigi yogo."
4,opolopo ohun ni mo ri ko nibi ipade yen.,ogopogologopogo ogohungun nigi mogo rigi kogo ...


In [ ]:
test_data1.drop(columns=['English'], inplace=True)

KeyError: "['English'] not found in axis"

In [ ]:
test_data1.head()

,source_text,target_text
0,won ta ofa fun jagunjagun naa laya.,wongon taga ogofaga fungun jagagungunjagagungu...
1,o ni imo pupo ni opolo.,ogo nigi igimogo pugupogo nigi ogopogologo.
2,idi ti mo fi feran aye oko ni pe gbogbo nnkan ...,igidigi tigi mogo figi fegerangan agayege ogok...
3,"mama mi, mo ti yo.","magamaga migi, mogo tigi yogo."
4,opolopo ohun ni mo ri ko nibi ipade yen.,ogopogologopogo ogohungun nigi mogo rigi kogo ...


In [ ]:
# Assuming test_data is already loaded with 'source_text' and 'target_text' columns
translated_sentences = []

# Translate each sentence in the test dataset
for source in test_data1["source_text"]:
    translated_sentence = translate_text(source)  # Use the function you defined
    translated_sentences.append(translated_sentence)

# Add the translations to the test dataset for comparison
test_data1["translated_text"] = translated_sentences


In [ ]:
test_data1.head()

,source_text,target_text,translated_text
0,won ta ofa fun jagunjagun naa laya.,wongon taga ogofaga fungun jagagungunjagagungu...,won taga aga fungun jagungun jagungun nagaaga ...
1,o ni imo pupo ni opolo.,ogo nigi igimogo pugupogo nigi ogopogologo.,ogo nigi igirogo pagatogo nigi ogopologo.
2,idi ti mo fi feran aye oko ni pe gbogbo nnkan ...,igidigi tigi mogo figi fegerangan agayege ogok...,igidigi tigi mogo figi wagawagbogo agayege oko...
3,"mama mi, mo ti yo.","magamaga migi, mogo tigi yogo.","mamani migi, mogo tigi yogo."
4,opolopo ohun ni mo ri ko nibi ipade yen.,ogopogologopogo ogohungun nigi mogo rigi kogo ...,ogopolopolopogo ogohungun nigi mogo rigi kogo ...


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Prepare reference and candidate lists
references = [[target.split()] for target in test_data1["target_text"]]
predictions = [pred.split() for pred in test_data1["translated_text"]]

# Calculate BLEU score
bleu_score = corpus_bleu(references, predictions)
print(f"Corpus BLEU Score: {bleu_score:.2f}")


Corpus BLEU Score: 0.45


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=df66380da26934117d1dd494176e19dadb45c4879082454cd76117c8d2cfad06
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import rouge_score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Evaluate each translation
rouge_scores = []
for ref, pred in zip(test_data1["target_text"], test_data1["translated_text"]):
    scores = scorer.score(ref, pred)
    rouge_scores.append(scores)

# Example: Print average ROUGE-1 scores
avg_rouge1 = sum(score["rouge1"].fmeasure for score in rouge_scores) / len(rouge_scores)
print(f"Average ROUGE-1 F1 Score: {avg_rouge1:.2f}")


Average ROUGE-1 F1 Score: 0.67
